In [1]:
#Import dependencies for random numbers
import numpy as np

#import dependencies to get city based on latitude and longitude
from citipy import citipy

#for API
import sys
sys.path.append("../")
import requests
from config import weather_api_key

# Import the time library and the datetime module from the datetime library
# Will need these for logging purposes 
import time
from datetime import datetime


In [2]:
#Create a set of 2,000 random latitudes and longitudes
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitude and logitudes to a list
# Note. we can only unzip a zipped tuple once before it is m oved from the comp's memory
coordinates = list(lat_lngs)
print(f"Number of coordinates: {len(coordinates)}")
coordinates[:10]

Number of coordinates: 2000


[(-35.65365785633099, 35.39964955392966),
 (56.34812372418489, 85.93935086784097),
 (-69.72083260545418, -68.33297253209207),
 (-52.21788519743825, -42.43812944604977),
 (-3.334561515297054, 70.13136470520712),
 (-82.70813240652976, -134.21785509073896),
 (-58.6286167291281, 73.99116494592226),
 (65.34205336881882, -138.0284646790343),
 (80.5981323726358, 110.95709746934011),
 (-30.128668335427733, 118.08727815046086)]

In [4]:
# Get the nearest city using citypy module
# Create a list for holding the cities
cities = []

# Identify the nearest city for each coordinate in the coordiantes list
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    #Do not have repeated cities in the cities list.
    if city not in cities:
        cities.append(city)

print(f"Number of cities: {len(cities)}")
cities[:5]


Number of cities: 755


['margate', 'anzhero-sudzhensk', 'ushuaia', 'mar del plata', 'hithadhoo']

In [5]:
# Perform an API call with the OpenWeatherMap in the next steps.

# Starting URL for Weather Map API Call.
weather_base_url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [6]:
# Understand the citipy returned data for just one city
city_url = weather_base_url + "&q=" + "Houston"
houston_weather = requests.get(city_url).json()
houston_weather


{'coord': {'lon': -95.3633, 'lat': 29.7633},
 'weather': [{'id': 721,
   'main': 'Haze',
   'description': 'haze',
   'icon': '50d'}],
 'base': 'stations',
 'main': {'temp': 76.98,
  'feels_like': 78.01,
  'temp_min': 72.9,
  'temp_max': 81.64,
  'pressure': 1011,
  'humidity': 77},
 'visibility': 6437,
 'wind': {'speed': 19.57, 'deg': 170, 'gust': 32.21},
 'clouds': {'all': 100},
 'dt': 1646604968,
 'sys': {'type': 2,
  'id': 2001415,
  'country': 'US',
  'sunrise': 1646570508,
  'sunset': 1646612632},
 'timezone': -21600,
 'id': 4699066,
 'name': 'Houston',
 'cod': 200}

In [7]:
# Test - Retrieve relevant fields just for one city

lat = houston_weather["coord"]["lat"]
lng = houston_weather["coord"]["lon"]
max_temp = houston_weather["main"]["temp_max"]
humidity = houston_weather["main"]["humidity"]
clouds = houston_weather["clouds"]["all"]
wind = houston_weather["wind"]["speed"]
country = houston_weather["sys"]["country"]
description = houston_weather["weather"][0]["description"]

print(lat, lng, max_temp, humidity, clouds, wind, country, description)


29.7633 -95.3633 81.64 77 100 19.57 US haze


In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = weather_base_url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1

# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_description,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | margate
Processing Record 2 of Set 1 | anzhero-sudzhensk
Processing Record 3 of Set 1 | ushuaia
Processing Record 4 of Set 1 | mar del plata
Processing Record 5 of Set 1 | hithadhoo
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | saint-philippe
Processing Record 8 of Set 1 | mayo
Processing Record 9 of Set 1 | saskylakh
Processing Record 10 of Set 1 | northam
Processing Record 11 of Set 1 | birin
Processing Record 12 of Set 1 | samusu
City not found. Skipping...
Processing Record 13 of Set 1 | cape town
Processing Record 14 of Set 1 | kahului
Processing Record 15 of Set 1 | yaan
Processing Record 16 of Set 1 | arraial do cabo
Processing Record 17 of Set 1 | provideniya
Processing Record 18 of Set 1 | barrow
Processing Record 19 of Set 1 | mataura
Processing Record 20 of Set 1 | tautira
Processing Record 21 of Set 1 | mahebourg
Processing Record 22 of Set 1 | narsaq
Processin

In [9]:
print(f"city_data has {len(city_data)} elements")

city_data has 686 elements


In [10]:
#Import pandas to create a dataframe
import pandas as pd

In [11]:
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,Margate,GB,51.3813,1.3862,42.96,66,29,5.99,scattered clouds,2022-03-06 22:18:12
1,Anzhero-Sudzhensk,RU,56.0810,86.0285,33.46,97,100,14.79,snow,2022-03-06 22:18:12
2,Ushuaia,AR,-54.8000,-68.3000,42.42,87,75,5.75,broken clouds,2022-03-06 22:17:57
3,Mar Del Plata,AR,-38.0023,-57.5575,69.10,90,20,11.99,few clouds,2022-03-06 22:18:13
4,Hithadhoo,MV,-0.6000,73.0833,82.35,70,29,10.04,scattered clouds,2022-03-06 22:18:13
5,Rikitea,PF,-23.1203,-134.9692,79.29,77,100,18.19,overcast clouds,2022-03-06 22:18:14
6,Saint-Philippe,RE,-21.3585,55.7679,78.39,87,100,16.71,light rain,2022-03-06 22:18:14
7,Mayo,US,38.8876,-76.5119,77.04,58,0,9.22,clear sky,2022-03-06 22:18:15
8,Saskylakh,RU,71.9167,114.0833,-12.78,100,99,1.48,overcast clouds,2022-03-06 22:18:15
9,Northam,GB,51.0333,-4.2167,41.40,78,28,11.65,scattered clouds,2022-03-06 22:18:15


In [12]:
# Drop the Date column of the city_data_df
city_data_df = city_data_df.iloc[:, :-1]
# city_data_df.reset_index(drop=True, inplace=True)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Margate,GB,51.3813,1.3862,42.96,66,29,5.99,scattered clouds
1,Anzhero-Sudzhensk,RU,56.0810,86.0285,33.46,97,100,14.79,snow
2,Ushuaia,AR,-54.8000,-68.3000,42.42,87,75,5.75,broken clouds
3,Mar Del Plata,AR,-38.0023,-57.5575,69.10,90,20,11.99,few clouds
4,Hithadhoo,MV,-0.6000,73.0833,82.35,70,29,10.04,scattered clouds
5,Rikitea,PF,-23.1203,-134.9692,79.29,77,100,18.19,overcast clouds
6,Saint-Philippe,RE,-21.3585,55.7679,78.39,87,100,16.71,light rain
7,Mayo,US,38.8876,-76.5119,77.04,58,0,9.22,clear sky
8,Saskylakh,RU,71.9167,114.0833,-12.78,100,99,1.48,overcast clouds
9,Northam,GB,51.0333,-4.2167,41.40,78,28,11.65,scattered clouds


In [13]:
# Export dataframe as a csv file and save it in the weather_database folder

# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"

# Export the city_data_df into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")